# Les optimisations multipériodes.

# Introduction

Les optimisations multipériodes sont effectuées sur un intervalle de périodes. 
Par exemple, les 12 mois de l'années, ou les 7 jours de la semaine.
L'optimisation la plus connue est l'optimisation de production multipériode linéaire qui permet d'équilibrer les stocks et la production.

Etude globale proposée par <b>Estelle Derrien - Github Estelle15000</b>

# Sommaire

- 1. Optimisation de production multipériode.
        - Notre problème de base donné par Taïwan university
        - Modélisation mathématique
        - Solution avec Python Pulp
           - Sans itérations
           - Explication de la solution
           - Avec itérations
- 2. Optimisation de production multipériode.
        - Notre problème de base donné par Columbia university
        - Modélisation mathématique
        - Solution avec Python Pulp
            - Avec itérations
            - Explication de la solution
            - Comparaison avec le résultat du solveur Excel
- 2. Affectation d'employés multipériode.
        - Notre problème de base
        - Modélisation mathématique
        - Résolution


# 1 Optimisation de production multipériode.

# Notre problème de base.

HISTOIRE COMPLETE TIREE DU COURS DE TAIWAN UNIVERSITY / COURSERA: 

Le manager prévoit qu'on va vendre 100 , 150, 200 et 170 les jours 1 ,2 , 3 et 4 de la prochaine semaine.

Les coûts de production varient en fonction du jour de la semaine: 
9e 12e 10e ET 12e les jours 1 ,2 , 3 et 4 de la prochaine semaine.

Le coût de stockage est de 1 euro par objet stocké donc, si on le garde 4 jours, ça fait 4 euros de coût. 

QUEL EST LE MEILLEUR PLAN DE PRODUCTION POUR MINIMISER LES COUTS ET NE PAS AVOIR TROP DE STOCKS ?

# Modélisation mathématique

<img src="img/01. Formulation du L.P.png"></img>

# Solution avec Pulp, sans itérations.

In [12]:
# Importer PULP
import pulp

# 1. On veut minimiser Les coûts de production + de stockage.
problem = pulp.LpProblem("Problem",pulp.LpMinimize)


# Les Variables de décision
# production d'objets par jour 1 , 2 , 3 et 4
P1 = pulp.LpVariable('Prodution1', lowBound=0, cat='Integer')
P2 = pulp.LpVariable('Prodution2', lowBound=0, cat='Integer')
P3 = pulp.LpVariable('Prodution3', lowBound=0, cat='Integer')
P4 = pulp.LpVariable('Prodution4', lowBound=0, cat='Integer')


# Stocks par jour 1 , 2 , 3 et 4
S1 = pulp.LpVariable('Stock1', lowBound=0, cat='Integer')
S2 = pulp.LpVariable('Stock2', lowBound=0, cat='Integer')
S3 = pulp.LpVariable('Stock3', lowBound=0, cat='Integer')
S4 = pulp.LpVariable('Stock4', lowBound=0, cat='Integer')

#Objective function : On veut minimiser Le cout de prod + Le cout de stockage
problem += (9*P1 + 12*P2 + 10*P3 + 12*P4) + (S1 + S2 + S3 + S4) 

# Les contraintes de balance d'inventaire - Inventory balancing constraints
problem += P1 - 100 == S1
problem += S1 + P2 - 150 == S2
problem += S2 + P3 - 200 == S3
problem += S3 + P4 - 170 == S4

problem += P1 >= 0
problem += P2 >= 0 
problem += P3 >= 0 
problem += P4 >= 0 


# On résouds 
result = problem.solve()

#On imprime les résultats
print("---Premier jour---")
print("Production:" ,pulp.value(P1))
print("Stock:" ,pulp.value(S1))


print("---2nd jour---")
print("Production:" ,pulp.value(P2))
print("Stock:" ,pulp.value(S2))

print("---3ème jour---")
print("Production3:" ,pulp.value(P3))
print("Stock:" ,pulp.value(S3))

print("---4eme jour---")
print("Production:" ,pulp.value(P4))
print("Stock:" ,pulp.value(S4))

print("Cout global:" ,pulp.value(problem.objective))



---Premier jour---
Production: 250.0
Stock: 150.0
---2nd jour---
Production: 0.0
Stock: 0.0
---3ème jour---
Production3: 370.0
Stock: 170.0
---4eme jour---
Production: 0.0
Stock: 0.0
Cout global: 6270.0


# Explication du résultat : 

Notre intervalle multipériode est ici de 4 jours.

Le solveur calcule ce qu'il faut produire et mettre en stock, afin de se retrouver à la fin de ces 4 jours sans stock sur les bras, tout en satisfaisant la demande.

Le premier jour, on produit 250, ce qui convient aux prévisions du manager qui prévoit de vendre 100 la première journée, et on garde 150 en stock pour le deuxième jour.

Le troisième jour, on produit 370 toujours dans l'optique de convenir aux prévisions du manager, pour les 2 jours suivant, et on garde 170 unités en stock.

Le quatrième jour, on a miniisé notre stock et notre coôut de stockage et il ne nous reste plus rien.

# Solution avec Pulp, avec itérations.



In [13]:
# Importer PULP
from pulp import *

# 1. On veut minimiser Les coûts de production, et d'inventaire.
model = LpProblem("Minimiser_le_cout_production",LpMinimize)

# ON définit les couts de production et de stockage pour les jours 1 2 3 ET 4 ; Dans quaters, c'est le nombre de jours SOIT 4. Define production cost, inventory cost, and demande.
# Définir les couts de production par période, le cout de l'inventaire, et la demandee reportée par le service commercial
quaters = list(range(4))
cout_production=[9,12,10,12]
cout_stockage=[1,1,1,1]
demande=[100,150,200,170]


# 2. Définir les variables de décision : Production et Stock - Define Decision Variables: Production and Inventory
x = LpVariable.dicts('quater_prod_', quaters,lowBound=0, cat='Continuous')
y = LpVariable.dicts('quater_stock_', quaters,lowBound=0, cat='Continuous')


# 3. Définir l'objectif , On veut minimiser les coûts de production + de stockage comme dans le problème de université taiwan
model += lpSum([cout_production[i]*x[i] for i in quaters]) + lpSum([cout_stockage[i]*y[i] for i in quaters])

# Définir les contraintes
# Constrainte de capacité de production (Production-capacity constraints)
for i in quaters:
    model.addConstraint(x[i]<=3000)

# Contrainte de balance de stocks ( Inventory-balance constraints)
model.addConstraint(x[0] - y[0] == demande[0]) # (Month 1)

for i in quaters[1:]:
    model.addConstraint(x[i] - y[i] + y[i-1] == demande[i]) # par (jour 2, 3, 4) 

#ON résouds avec le solveur pulp ou un autre entre parenthèses
model.solve()

# On imprime les solutions
for v in model.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Notre fonction objectif , le coût global minimisé =  ", value(model.objective))


quater_prod__0 = 250.0
quater_prod__1 = 0.0
quater_prod__2 = 370.0
quater_prod__3 = 0.0
quater_stock__0 = 150.0
quater_stock__1 = 0.0
quater_stock__2 = 170.0
quater_stock__3 = 0.0
Notre fonction objectif , le coût global minimisé =   6270.0
